In [1]:
import os, json, random, torch, sys
import numpy as np
from glob import glob
from PIL import Image
from tqdm import tqdm
import torch.nn.functional as F
#from torchsummary import summary
from matplotlib import pyplot as plt
from multiprocessing import Pool, cpu_count
import ipywidgets as ipw
sys.path.append("../")

from utils import image
from utils.data import NumpyImageLoader
from utils.learning import get_best_checkpoint

In [2]:
pools = Pool(processes=cpu_count())

In [3]:
from utils.MobileNetV2_pretrained_imagenet import MobileNetV2

params = {}
params["channel"] = "YCbCr"
params["threshold"] = 0.65
params["training_log_dir"] = "backup/MBN2-YCbCr/checkpoints/"

MODEL_FILE = os.path.join(params["training_log_dir"], "model.ckpt")
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = MobileNetV2(n_class=2, input_size=64, width_mult=1.0).to(device=DEVICE)
model.load(model_file=MODEL_FILE)

/home/ubuntu/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/torch/serialization.py:650: SourceChangeWarning: source code of class 'torch.nn.modules.loss.CrossEntropyLoss' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [4]:
def createUploadButton():
    myimage = ipw.FileUpload(
        accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
        multiple=False  # True to accept multiple files upload else False
    )
    return myimage

In [5]:
def evaluate_image(uploaded_filename):
    IMAGE_FILE = uploaded_filename
    GT = "Positive" 
    img_PIL = Image.open(IMAGE_FILE).convert("RGB")
    img = np.array(Image.open(IMAGE_FILE).convert(params["channel"]))

    coords, _, _ = image.slide2d(sz=img.shape[:2], K=64, S=32)
    patches = image.crop_patches( img=img, coords=coords, patch_sz=64)
    loader = NumpyImageLoader(ndarray_data=patches, batch_size=16, 
                              n_workers=cpu_count(), pin_memory=True, shuffle=False).loader

    # Predict
    softmaxs = []
    model.eval()
    for X in loader:
        X = X[0].to(DEVICE)
        logits = model(X)
        softmaxs.append(F.softmax(logits, dim=1).detach().cpu().numpy())
    softmaxs = np.concatenate(softmaxs, axis=0)

    # Post-processing
    labels = image.post_process(softmaxs[:,1], coords, 8, params["threshold"], 32, pools=pools)
    decision = image.fusion(labels)
    
    fig=plt.figure(figsize=(18, 16), dpi= 80, facecolor='w', edgecolor='k')
    plt.imshow(img_PIL); plt.title(uploaded_filename); plt.axis('off')
    if decision==1:
        return "Fake"
    else:
        return "Real Image"

In [16]:
output = ipw.Text(placeholder="----", layout=ipw.Layout(width="190px"),  disabled=False, description='Prediction')
from IPython.display import clear_output
from ipywidgets import IntSlider, Output
from IPython.display import display

def on_click(btn):
    if btn.description == "Evaluate":
        try:
            uploaded_filename = next(iter(myimage.value))            
            content = myimage.value[uploaded_filename]['content']
            with open(uploaded_filename, 'wb') as f: 
                f.write(content)
            output.value = evaluate_image(uploaded_filename)
            
        except:
            output.value = "ERROR"
    elif btn.description == "Clear":
        myimage.values = {}
        output.value = ""
        clear_output(wait=False)
        display(pred_op)

    elif btn.description == "del":
        output.value = output.value[:-1]
        
    else:
        output.value = output.value + btn.description

def mk_btn(description):
    btn = ipw.Button(description=description, layout=ipw.Layout(width="100px" , height="30px"))
    btn.on_click(on_click)
    return btn



myimage = createUploadButton()
label = ipw.Label(value='Test for document authenticity')
row0 = ipw.HBox([mk_btn("Evaluate") , mk_btn("Clear")])
pred_op = ipw.VBox((label, myimage, row0,  output))
display(pred_op)